In [1]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


### Step 2: Loading the Dataset

In [2]:
file_path = '2206MCPC_VA (1).xlsx'
df = pd.read_excel(file_path)

List the columns

In [3]:
list(df.columns)

['Age',
 'Age Group',
 'Ethnicity',
 'Race',
 'Clinic assigned to',
 'Zip Code',
 'City',
 'Year',
 'Text Follow up Prior Scheduling Completed PrEP Appointment',
 'Call Follow up Prior Scheduling Completed PrEP Appointment',
 'Email Follow up Prior Scheduling Completed PrEP Appointment',
 'Other Follow up Prior Scheduling Completed PrEP Appointment',
 'Text Follow up Between Scheduling and Completed PrEP Appointment Date',
 'Call Follow up Between Scheduling and Completed PrEP Appointment Date',
 'Email Follow up Between Scheduling and Completed PrEP Appointment Date',
 'Other Follow up Between Scheduling and Completed PrEP Appointment Date',
 'First Completed PrEP Appointment Scheduled Date',
 'First Completed PrEP Appointment Date',
 'Telehealth type',
 'Number of Previous Incomplete appointments',
 'Previous Incomplete appointments type',
 'Provider',
 'Timeslot',
 'Day of week',
 'Month',
 'Waiting time(Days)',
 'Time spent at clinic(Min)',
 'First Filled Date After Completed Appoi

### Step 3: Handling Missing Values

In [4]:
# Fill missing values in 'Zip Code' and 'City'
df['Zip Code'].fillna(df['Zip Code'].mode()[0], inplace=True)
df['City'].fillna(df['City'].mode()[0], inplace=True)


# Fill missing values in 'Provider' with 'Unknown'
df['Provider'].fillna('Unknown', inplace=True)

# Fill missing values in 'Time spent at clinic(Min)' with the median
df['Time spent at clinic(Min)'].fillna(df['Time spent at clinic(Min)'].median(), inplace=True)


C:\Users\PLC Group\AppData\Local\Temp\ipykernel_20448\702166628.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Zip Code'].fillna(df['Zip Code'].mode()[0], inplace=True)
C:\Users\PLC Group\AppData\Local\Temp\ipykernel_20448\702166628.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves 

The missing values (or nulls) in the Zip Code and City columns are replaced with the mode. The missing values in the Provider column are filled with 'Unknown'. The missing values in the 'Time spent at clinic(Min)' column are filled with median.

### Step 4: Feature Engineering

In [5]:
# Create a binary indicator for latefill
df['latefill'] = df['Fill shipment waiting period'].apply(lambda x: 1 if x > 30 else 0)

# Convert date columns to datetime format
df['First Completed PrEP Appointment Scheduled Date'] = pd.to_datetime(df['First Completed PrEP Appointment Scheduled Date'])
df['First Completed PrEP Appointment Date'] = pd.to_datetime(df['First Completed PrEP Appointment Date'])

# Add 'Appointment_Month' column
df['Appointment_Month'] = df['First Completed PrEP Appointment Date'].dt.month

# Convert 'First Filled Date After Completed Appointment' to datetime and create a new feature
df['First Filled Date After Completed Appointment'] = pd.to_datetime(df['First Filled Date After Completed Appointment'])
df['First_Filled_Days_Since'] = (df['First Filled Date After Completed Appointment'] - df['First Filled Date After Completed Appointment'].min()).dt.days


Binary Indicator (latefill) Creation: A new binary column latefill has been created based on the 'Fill shipment waiting period' column. It indicates whether the waiting period was more than 30 days (1) or not (0).
Date Conversion: The date columns 'First Completed PrEP Appointment Scheduled Date' and 'First Completed PrEP Appointment Date' have been successfully converted to datetime format.
Displaying the Result: The output shows the latefill column along with the converted date columns, demonstrating the transformations.

### Step 5: Further Feature Engineering and Dropping Columns

In [6]:
# Add 'Appointment_Month' column
df['Appointment_Month'] = df['First Completed PrEP Appointment Date'].dt.month

# Convert 'First Filled Date After Completed Appointment' to datetime and create a new feature
df['First Filled Date After Completed Appointment'] = pd.to_datetime(df['First Filled Date After Completed Appointment'])
df['First_Filled_Days_Since'] = (df['First Filled Date After Completed Appointment'] - df['First Filled Date After Completed Appointment'].min()).dt.days

# Drop the original 'First Filled Date After Completed Appointment' column
df.drop(columns=['First Filled Date After Completed Appointment'], inplace=True)

df[['Appointment_Month', 'First_Filled_Days_Since']].head()


,Appointment_Month,First_Filled_Days_Since
0,6,707.0
1,6,714.0
2,6,709.0
3,8,1065.0
4,8,NaN


A new column Appointment_Month has been added, representing the month of the 'First Completed PrEP Appointment Date'. The First_Filled_Days_Since column calculates the number of days since the earliest 'First Filled Date After Completed Appointment'. The original 'First Filled Date After Completed Appointment' column has been dropped. The output shows the new Appointment_Month and First_Filled_Days_Since columns.

### Step 6: Processing Zip Code and Encoding Categorical Variables

In [7]:
# Convert 'Zip Code' to integer after filtering out non-digit characters
df['Zip Code'] = df['Zip Code'].apply(lambda x: int(''.join(filter(str.isdigit, str(x)))))

# One-hot encode categorical columns
df_encoded = pd.get_dummies(df, columns=[
    'Age Group', 'Ethnicity', 'Race', 'Clinic assigned to', 'Telehealth type', 
    'Previous Incomplete appointments type', 'Provider', 'Timeslot', 
    'Day of week', 'Appointment_Month'
])

df_encoded.head()


,Age,Zip Code,City,Year,Text Follow up Prior Scheduling Completed PrEP Appointment,Call Follow up Prior Scheduling Completed PrEP Appointment,Email Follow up Prior Scheduling Completed PrEP Appointment,Other Follow up Prior Scheduling Completed PrEP Appointment,Text Follow up Between Scheduling and Completed PrEP Appointment Date,Call Follow up Between Scheduling and Completed PrEP Appointment Date,...,Appointment_Month_3,Appointment_Month_4,Appointment_Month_5,Appointment_Month_6,Appointment_Month_7,Appointment_Month_8,Appointment_Month_9,Appointment_Month_10,Appointment_Month_11,Appointment_Month_12
0,22,37129,MURFREESBORO,2021,0,0,0,0,0,0,...,False,False,False,True,False,False,False,False,False,False
1,38,37205,NASHVILLE,2021,0,0,0,0,0,0,...,False,False,False,True,False,False,False,False,False,False
2,34,75204,DALLAS,2021,0,0,0,0,0,0,...,False,False,False,True,False,False,False,False,False,False
3,25,37211,NASHVILLE,2021,0,0,0,0,0,0,...,False,False,False,False,False,True,False,False,False,False
4,28,372074842,NASHVILLE,2021,0,0,0,0,0,0,...,False,False,False,False,False,True,False,False,False,False


Processing Zip Code: The 'Zip Code' column has been converted to integers by removing any non-digit characters.
One-Hot Encoding: Categorical columns, such as 'Age Group', 'Ethnicity', 'Race', etc., have been one-hot encoded. This creates new binary columns for each category, resulting in 135 columns in the encoded DataFrame.
Displaying the Encoded Data: The first few rows of the one-hot encoded DataFrame are displayed, showing the newly created columns.

### Step: Remove the Insurance column

In [8]:
list(df_encoded.columns)

['Age',
 'Zip Code',
 'City',
 'Year',
 'Text Follow up Prior Scheduling Completed PrEP Appointment',
 'Call Follow up Prior Scheduling Completed PrEP Appointment',
 'Email Follow up Prior Scheduling Completed PrEP Appointment',
 'Other Follow up Prior Scheduling Completed PrEP Appointment',
 'Text Follow up Between Scheduling and Completed PrEP Appointment Date',
 'Call Follow up Between Scheduling and Completed PrEP Appointment Date',
 'Email Follow up Between Scheduling and Completed PrEP Appointment Date',
 'Other Follow up Between Scheduling and Completed PrEP Appointment Date',
 'First Completed PrEP Appointment Scheduled Date',
 'First Completed PrEP Appointment Date',
 'Number of Previous Incomplete appointments',
 'Month',
 'Waiting time(Days)',
 'Time spent at clinic(Min)',
 'First Shipped Date After Completed Appointment',
 'Insurance Status(First Filled Date After Completed Appointment)',
 'Insurance Status Source(First Filled Date After Completed Appointment)',
 'Fill ship

In [9]:
for column in df_encoded.columns:
    if 'insurance' in column.lower():
        print(f"Removing column: {column}")
        df_encoded.drop(column, axis=1, inplace=True)

Removing column: Insurance Status(First Filled Date After Completed Appointment)
Removing column: Insurance Status Source(First Filled Date After Completed Appointment)


### Step 7: Create Target Variable and Clean Up DataFrame

In [10]:
# Ensure 'Shipment_Occurred' is correctly created before encoding
df_encoded['Shipment_Occurred'] = df['First Shipped Date After Completed Appointment'].apply(
    lambda x: 1 if pd.notna(x) else 0
)

# Identify columns with non-numeric data that should be numeric
non_numeric_columns = df_encoded.select_dtypes(exclude=[np.number]).columns

# Drop the non-numeric columns that cannot be converted directly
df_encoded_cleaned = df_encoded.drop(columns=non_numeric_columns)

df_encoded_cleaned.head()


,Age,Zip Code,Year,Text Follow up Prior Scheduling Completed PrEP Appointment,Call Follow up Prior Scheduling Completed PrEP Appointment,Email Follow up Prior Scheduling Completed PrEP Appointment,Other Follow up Prior Scheduling Completed PrEP Appointment,Text Follow up Between Scheduling and Completed PrEP Appointment Date,Call Follow up Between Scheduling and Completed PrEP Appointment Date,Email Follow up Between Scheduling and Completed PrEP Appointment Date,Other Follow up Between Scheduling and Completed PrEP Appointment Date,Number of Previous Incomplete appointments,Waiting time(Days),Time spent at clinic(Min),Fill shipment waiting period,latefill,First_Filled_Days_Since,Shipment_Occurred
0,22,37129,2021,0,0,0,0,0,0,0,0,0,3.0,67.0,1.0,0,707.0,1
1,38,37205,2021,0,0,0,0,0,0,0,0,0,3.0,74.0,8.0,0,714.0,1
2,34,75204,2021,0,0,0,0,0,0,0,0,0,0.0,67.0,5.0,0,709.0,1
3,25,37211,2021,0,0,0,0,0,0,0,0,0,2.0,80.0,304.0,1,1065.0,1
4,28,372074842,2021,0,0,0,0,0,0,0,0,0,0.0,54.0,NaN,0,NaN,0


Creating the Target Variable (Shipment_Occurred): A binary target variable Shipment_Occurred was created, indicating whether a shipment occurred (1) or not (0).
Cleaning the DataFrame: Non-numeric columns that couldn't be converted directly were identified and dropped from the DataFrame.
Displaying the Cleaned DataFrame: The cleaned DataFrame now has 127 columns, and the first few rows are displayed, including the newly added Shipment_Occurred column.

### Step 8: Imputation of Missing Values

In [11]:
from sklearn.impute import SimpleImputer

# Re-impute missing values for numeric columns
numeric_cols = df_encoded_cleaned.select_dtypes(include=[np.number]).columns
categorical_cols = df_encoded_cleaned.select_dtypes(exclude=[np.number]).columns

imputer_numeric = SimpleImputer(strategy='median')
df_encoded_cleaned[numeric_cols] = imputer_numeric.fit_transform(df_encoded_cleaned[numeric_cols])

# Fill missing values in categorical columns manually with the mode (most frequent value)
for column in categorical_cols:
    df_encoded_cleaned[column].fillna(df_encoded_cleaned[column].mode()[0], inplace=True)

df_encoded_cleaned.head()


,Age,Zip Code,Year,Text Follow up Prior Scheduling Completed PrEP Appointment,Call Follow up Prior Scheduling Completed PrEP Appointment,Email Follow up Prior Scheduling Completed PrEP Appointment,Other Follow up Prior Scheduling Completed PrEP Appointment,Text Follow up Between Scheduling and Completed PrEP Appointment Date,Call Follow up Between Scheduling and Completed PrEP Appointment Date,Email Follow up Between Scheduling and Completed PrEP Appointment Date,Other Follow up Between Scheduling and Completed PrEP Appointment Date,Number of Previous Incomplete appointments,Waiting time(Days),Time spent at clinic(Min),Fill shipment waiting period,latefill,First_Filled_Days_Since,Shipment_Occurred
0,22.0,37129.0,2021.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,67.0,1.0,0.0,707.0,1.0
1,38.0,37205.0,2021.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,74.0,8.0,0.0,714.0,1.0
2,34.0,75204.0,2021.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67.0,5.0,0.0,709.0,1.0
3,25.0,37211.0,2021.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,80.0,304.0,1.0,1065.0,1.0
4,28.0,372074842.0,2021.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54.0,9.0,0.0,986.0,0.0


Section 6: Explanation of the Output

Numeric Columns Imputation: Missing values in the numeric columns were re-imputed using the median strategy.
Categorical Columns Imputation: Missing values in categorical columns (if any were left) were filled with the mode (most frequent value).
Displaying the Imputed DataFrame: The DataFrame now shows the imputed values, ensuring no missing values remain in the dataset.

### Step: Outlier Analysis and Removal

In [14]:
from sklearn.ensemble import IsolationForest

# Analyze which columns have outliers
numeric_columns = df_encoded_cleaned.select_dtypes(include=[np.number]).columns
numeric_columns = numeric_columns.drop('Shipment_Occurred')
outlier_summary = {}

for column in numeric_columns:
    iso_forest = IsolationForest(contamination=0.05, random_state=42)
    outliers = iso_forest.fit_predict(df_encoded_cleaned[[column]])
    outlier_count = sum(outliers == -1)
    outlier_summary[column] = outlier_count

# Convert the summary to a DataFrame for better visualization
outlier_summary_df = pd.DataFrame(list(outlier_summary.items()), columns=['Column', 'Outlier Count'])
outlier_summary_df.sort_values(by='Outlier Count', ascending=False, inplace=True)

Zip Code (519 outliers):

Approach: Outliers in Zip Code might indicate errors or non-standard entries. These could be replaced with the mode (most frequent value) or dropped if they significantly deviate from valid zip codes.

Fill shipment waiting period (516 outliers):

Approach: Since this column likely represents the number of days a shipment was delayed, outliers may represent extreme delays. Consider capping the outliers (i.e., replacing extreme values with a maximum threshold) or analyzing if these extreme cases are meaningful and should be kept.

Time spent at clinic (Min) (513 outliers):

Approach: Extreme values might indicate unusually long or short clinic visits. Outliers can be handled by capping or using the median to replace them if the extremes are not meaningful.

First_Filled_Days_Since (512 outliers):

Approach: This feature tracks the days since the first filled date, and outliers might represent very early or very late fills. Capping could be appropriate here, or outliers could be left if they are meaningful in the context of your analysis.

Email Follow up Prior Scheduling Completed PrEP Appointment (497 outliers):

Approach: Outliers could represent an unusually high number of follow-ups. Depending on the distribution, consider capping or replacing outliers with the median.

Text Follow up Prior Scheduling Completed PrEP Appointment (471 outliers):

Approach: Similar to email follow-ups, consider capping or using the median to replace outliers.

Waiting time (Days) (445 outliers):

Approach: Long or short waiting times could be outliers. Depending on the distribution and business rules, you could cap these values or transform them (e.g., log transformation) to handle the skewness.

Age (436 outliers):

Approach: Age outliers might indicate either very young or very old patients. If these values are biologically implausible (e.g., very young children), they should be treated as errors and either corrected or removed.

Other Follow up Prior Scheduling Completed PrEP Appointment (402 outliers):

Approach: Handle similarly to other follow-up columns by capping or replacing outliers.

Number of Previous Incomplete appointments (390 outliers):

Approach: High numbers of incomplete appointments might be outliers. These could be capped or treated with a transformation.

Text Follow up Between Scheduling and Completed Appointment (368 outliers):

Approach: Similar approach as other follow-up columns, consider capping or replacing with the median.

Call Follow up Prior Scheduling Completed PrEP Appointment (339 outliers):

Approach: Handle similarly to other follow-up columns.

Call Follow up Between Scheduling and Completed Appointment (231 outliers):

Approach: Similar to other follow-up columns, consider capping or median replacement.

Other Follow up Between Scheduling and Completed Appointment (83 outliers):

Approach: With fewer outliers, consider reviewing the data distribution and capping or replacing as needed.

Email Follow up Between Scheduling and Completed Appointment (17 outliers):

Approach: Handle similarly to the above follow-up columns.

Year (4 outliers):

Approach: Outliers in Year could represent incorrect data entries (e.g., future years). These should be corrected or removed.

latefill (0 outliers):

Approach: No outliers detected, so no handling is necessary for this column.

In [15]:
# Handle outliers in each column separately according to the discussed strategies

# Zip Code: Replace outliers with the mode
zip_code_mode = df_encoded_cleaned['Zip Code'].mode()[0]
df_encoded_cleaned['Zip Code'] = df_encoded_cleaned['Zip Code'].apply(
    lambda x: zip_code_mode if x in IsolationForest(contamination=0.05, random_state=42).fit_predict(df_encoded_cleaned[['Zip Code']]) else x
)

# Fill shipment waiting period: Cap the outliers
fill_shipment_waiting_period_cap = df_encoded_cleaned['Fill shipment waiting period'].quantile(0.95)
df_encoded_cleaned['Fill shipment waiting period'] = np.where(
    df_encoded_cleaned['Fill shipment waiting period'] > fill_shipment_waiting_period_cap,
    fill_shipment_waiting_period_cap,
    df_encoded_cleaned['Fill shipment waiting period']
)

# Time spent at clinic(Min): Cap the outliers
time_spent_clinic_cap = df_encoded_cleaned['Time spent at clinic(Min)'].quantile(0.95)
df_encoded_cleaned['Time spent at clinic(Min)'] = np.where(
    df_encoded_cleaned['Time spent at clinic(Min)'] > time_spent_clinic_cap,
    time_spent_clinic_cap,
    df_encoded_cleaned['Time spent at clinic(Min)']
)

# First_Filled_Days_Since: Cap the outliers
first_filled_days_since_cap = df_encoded_cleaned['First_Filled_Days_Since'].quantile(0.95)
df_encoded_cleaned['First_Filled_Days_Since'] = np.where(
    df_encoded_cleaned['First_Filled_Days_Since'] > first_filled_days_since_cap,
    first_filled_days_since_cap,
    df_encoded_cleaned['First_Filled_Days_Since']
)

# Email Follow up Prior Scheduling Completed PrEP Appointment: Cap the outliers
email_follow_up_prior_cap = df_encoded_cleaned['Email Follow up Prior Scheduling Completed PrEP Appointment'].quantile(0.95)
df_encoded_cleaned['Email Follow up Prior Scheduling Completed PrEP Appointment'] = np.where(
    df_encoded_cleaned['Email Follow up Prior Scheduling Completed PrEP Appointment'] > email_follow_up_prior_cap,
    email_follow_up_prior_cap,
    df_encoded_cleaned['Email Follow up Prior Scheduling Completed PrEP Appointment']
)

# Text Follow up Prior Scheduling Completed PrEP Appointment: Cap the outliers
text_follow_up_prior_cap = df_encoded_cleaned['Text Follow up Prior Scheduling Completed PrEP Appointment'].quantile(0.95)
df_encoded_cleaned['Text Follow up Prior Scheduling Completed PrEP Appointment'] = np.where(
    df_encoded_cleaned['Text Follow up Prior Scheduling Completed PrEP Appointment'] > text_follow_up_prior_cap,
    text_follow_up_prior_cap,
    df_encoded_cleaned['Text Follow up Prior Scheduling Completed PrEP Appointment']
)

# Waiting time(Days): Cap the outliers
waiting_time_days_cap = df_encoded_cleaned['Waiting time(Days)'].quantile(0.95)
df_encoded_cleaned['Waiting time(Days)'] = np.where(
    df_encoded_cleaned['Waiting time(Days)'] > waiting_time_days_cap,
    waiting_time_days_cap,
    df_encoded_cleaned['Waiting time(Days)']
)

# Age: Cap the outliers
age_cap = df_encoded_cleaned['Age'].quantile(0.95)
df_encoded_cleaned['Age'] = np.where(
    df_encoded_cleaned['Age'] > age_cap,
    age_cap,
    df_encoded_cleaned['Age']
)

# Other Follow up Prior Scheduling Completed PrEP Appointment: Cap the outliers
other_follow_up_prior_cap = df_encoded_cleaned['Other Follow up Prior Scheduling Completed PrEP Appointment'].quantile(0.95)
df_encoded_cleaned['Other Follow up Prior Scheduling Completed PrEP Appointment'] = np.where(
    df_encoded_cleaned['Other Follow up Prior Scheduling Completed PrEP Appointment'] > other_follow_up_prior_cap,
    other_follow_up_prior_cap,
    df_encoded_cleaned['Other Follow up Prior Scheduling Completed PrEP Appointment']
)

# Number of Previous Incomplete appointments: Cap the outliers
num_prev_incomplete_appts_cap = df_encoded_cleaned['Number of Previous Incomplete appointments'].quantile(0.95)
df_encoded_cleaned['Number of Previous Incomplete appointments'] = np.where(
    df_encoded_cleaned['Number of Previous Incomplete appointments'] > num_prev_incomplete_appts_cap,
    num_prev_incomplete_appts_cap,
    df_encoded_cleaned['Number of Previous Incomplete appointments']
)

# Text Follow up Between Scheduling and Completed Appointment: Cap the outliers
text_follow_up_between_cap = df_encoded_cleaned['Text Follow up Between Scheduling and Completed Appointment'].quantile(0.95)
df_encoded_cleaned['Text Follow up Between Scheduling and Completed Appointment'] = np.where(
    df_encoded_cleaned['Text Follow up Between Scheduling and Completed Appointment'] > text_follow_up_between_cap,
    text_follow_up_between_cap,
    df_encoded_cleaned['Text Follow up Between Scheduling and Completed Appointment']
)

# Call Follow up Prior Scheduling Completed PrEP Appointment: Cap the outliers
call_follow_up_prior_cap = df_encoded_cleaned['Call Follow up Prior Scheduling Completed PrEP Appointment'].quantile(0.95)
df_encoded_cleaned['Call Follow up Prior Scheduling Completed PrEP Appointment'] = np.where(
    df_encoded_cleaned['Call Follow up Prior Scheduling Completed PrEP Appointment'] > call_follow_up_prior_cap,
    call_follow_up_prior_cap,
    df_encoded_cleaned['Call Follow up Prior Scheduling Completed PrEP Appointment']
)

# Call Follow up Between Scheduling and Completed Appointment: Cap the outliers
call_follow_up_between_cap = df_encoded_cleaned['Call Follow up Between Scheduling and Completed Appointment'].quantile(0.95)
df_encoded_cleaned['Call Follow up Between Scheduling and Completed Appointment'] = np.where(
    df_encoded_cleaned['Call Follow up Between Scheduling and Completed Appointment'] > call_follow_up_between_cap,
    call_follow_up_between_cap,
    df_encoded_cleaned['Call Follow up Between Scheduling and Completed Appointment']
)

# Other Follow up Between Scheduling and Completed Appointment: Cap the outliers
other_follow_up_between_cap = df_encoded_cleaned['Other Follow up Between Scheduling and Completed Appointment'].quantile(0.95)
df_encoded_cleaned['Other Follow up Between Scheduling and Completed Appointment'] = np.where(
    df_encoded_cleaned['Other Follow up Between Scheduling and Completed Appointment'] > other_follow_up_between_cap,
    other_follow_up_between_cap,
    df_encoded_cleaned['Other Follow up Between Scheduling and Completed Appointment']
)

# Email Follow up Between Scheduling and Completed Appointment: Cap the outliers
email_follow_up_between_cap = df_encoded_cleaned['Email Follow up Between Scheduling and Completed Appointment'].quantile(0.95)
df_encoded_cleaned['Email Follow up Between Scheduling and Completed Appointment'] = np.where(
    df_encoded_cleaned['Email Follow up Between Scheduling and Completed Appointment'] > email_follow_up_between_cap,
    email_follow_up_between_cap,
    df_encoded_cleaned['Email Follow up Between Scheduling and Completed Appointment']
)

# Year: Handle Year outliers by setting them to a plausible range
valid_year_range = (df_encoded_cleaned['Year'].min(), df_encoded_cleaned['Year'].max())
df_encoded_cleaned['Year'] = df_encoded_cleaned['Year'].apply(
    lambda x: valid_year_range[0] if x < valid_year_range[0] else (valid_year_range[1] if x > valid_year_range[1] else x)
)

# latefill has no outliers, so no handling is needed

### Step 9: Balancing the Dataset by Oversampling

In [ ]:
from sklearn.utils import resample

# Separate majority and minority classes
df_majority = df_encoded_cleaned[df_encoded_cleaned['Shipment_Occurred'] == 1]
df_minority = df_encoded_cleaned[df_encoded_cleaned['Shipment_Occurred'] == 0]

# Perform oversampling on the minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=len(df_majority),    # to match majority class
                                 random_state=42) # reproducible results

# Combine majority class with upsampled minority class
df_balanced = pd.concat([df_majority, df_minority_upsampled])

df_balanced['Shipment_Occurred'].value_counts()


Balancing the Dataset: The minority class (Shipment_Occurred = 0) was oversampled to match the number of samples in the majority class (Shipment_Occurred = 1). This results in a balanced dataset.
Class Distribution: The output shows that both classes now have 8,657 samples, ensuring that the dataset is balanced for modeling.

### Step 10: Feature Correlation Analysis

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Define features (X) and target (y) after balancing
X_balanced = df_balanced.drop(columns=['Shipment_Occurred'])
y_balanced = df_balanced['Shipment_Occurred']

# Calculate the Pearson correlation coefficient for the features
corr_matrix = df_balanced.corr()

# Visualize the correlation matrix using a heatmap
plt.figure(figsize=(20, 15))
sns.heatmap(corr_matrix, annot=False, cmap='coolwarm', linewidths=0.5)
plt.title("Correlation Matrix Heatmap")
plt.show()

corr_matrix['Shipment_Occurred'].sort_values(ascending=False).head(10)


Correlation Matrix Calculation: The Pearson correlation coefficients between the features and the target variable (Shipment_Occurred) have been calculated.
Heatmap Visualization: A heatmap of the correlation matrix is displayed, showing the strength and direction of relationships between variables.
Top Correlated Features: The top 10 features most positively correlated with the target variable Shipment_Occurred are displayed. The latefill feature has the highest positive correlation with the target.

### Step 11: Random Forest Model Fitting and Feature Importance

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Fit the RandomForest model
rf_balanced = RandomForestClassifier(random_state=42)
rf_balanced.fit(X_balanced, y_balanced)

# Calculate feature importances
importances_balanced = rf_balanced.feature_importances_
feature_importance_balanced = pd.Series(importances_balanced, index=X_balanced.columns)

# Sort features by importance
sorted_importance_balanced = feature_importance_balanced.sort_values(ascending=False)
sorted_importance_balanced.head(10)


### Step 12: Logistic Regression Model Evaluation

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Fit the Logistic Regression model
logreg_balanced = LogisticRegression(random_state=42, max_iter=1000)
logreg_balanced.fit(X_balanced, y_balanced)

# Predict on the training data
y_pred_balanced = logreg_balanced.predict(X_balanced)

# Evaluate the model
conf_matrix = confusion_matrix(y_balanced, y_pred_balanced)
class_report = classification_report(y_balanced, y_pred_balanced)
accuracy = accuracy_score(y_balanced, y_pred_balanced)

(conf_matrix, class_report, accuracy)


In [ ]:
# Neatly formatted classification report
formatted_class_report = classification_report(y_balanced, y_pred_balanced, target_names=['Class 0', 'Class 1'])

print("Confusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", formatted_class_report)
print("Accuracy:", accuracy)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Fit the RandomForest model
rf_balanced = RandomForestClassifier(random_state=42, n_estimators=100)
rf_balanced.fit(X_balanced, y_balanced)

# Predict on the training data
y_pred_rf_balanced = rf_balanced.predict(X_balanced)

# Evaluate the model
conf_matrix_rf = confusion_matrix(y_balanced, y_pred_rf_balanced)
class_report_rf = classification_report(y_balanced, y_pred_rf_balanced)
accuracy_rf = accuracy_score(y_balanced, y_pred_rf_balanced)

conf_matrix_rf, class_report_rf, accuracy_rf


In [ ]:
from sklearn.model_selection import train_test_split

# Split the balanced dataset into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42)

# Train the RandomForest model on the training set
rf_balanced_train = RandomForestClassifier(random_state=42, n_estimators=100)
rf_balanced_train.fit(X_train, y_train)

# Predict on the training set
y_pred_train = rf_balanced_train.predict(X_train)

# Predict on the test set
y_pred_test = rf_balanced_train.predict(X_test)

# Evaluate the model on the training set
train_accuracy = accuracy_score(y_train, y_pred_train)
train_class_report = classification_report(y_train, y_pred_train)

# Evaluate the model on the test set
test_accuracy = accuracy_score(y_test, y_pred_test)
test_class_report = classification_report(y_test, y_pred_test)

train_accuracy, test_accuracy, train_class_report, test_class_report


In [ ]:
from sklearn.model_selection import cross_val_score

# Perform 5-fold cross-validation on the balanced dataset
cv_scores = cross_val_score(RandomForestClassifier(random_state=42, n_estimators=100), X_balanced, y_balanced, cv=5, scoring='accuracy')

# Calculate mean and standard deviation of cross-validation scores
mean_cv_score = np.mean(cv_scores)
std_cv_score = np.std(cv_scores)

mean_cv_score, std_cv_score, cv_scores


### Reducing complexity and retraining to analyze overfitting

In [ ]:
# Train a less complex RandomForest model with limited max_depth and max_features
rf_limited = RandomForestClassifier(random_state=42, n_estimators=100, max_depth=5, max_features='sqrt')

# Fit the model on the training set
rf_limited.fit(X_train, y_train)

# Predict on the training and test sets
y_pred_train_limited = rf_limited.predict(X_train)
y_pred_test_limited = rf_limited.predict(X_test)

# Evaluate the model on the training set
train_accuracy_limited = accuracy_score(y_train, y_pred_train_limited)
train_class_report_limited = classification_report(y_train, y_pred_train_limited)

# Evaluate the model on the test set
test_accuracy_limited = accuracy_score(y_test, y_pred_test_limited)
test_class_report_limited = classification_report(y_test, y_pred_test_limited)

(train_accuracy_limited, test_accuracy_limited, train_class_report_limited, test_class_report_limited)


XGBoost

### Step: XGBOOST

In [ ]:
from xgboost import XGBClassifier

# Define the target and features
X_balanced = df_balanced.drop(columns=['Shipment_Occurred'])
y_balanced = df_balanced['Shipment_Occurred']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42)

# Apply XGBoost classifier
xgb_model = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)

# Predictions
y_pred_train = xgb_model.predict(X_train)
y_pred_test = xgb_model.predict(X_test)

# Evaluation
train_accuracy = accuracy_score(y_train, y_pred_train)
test_accuracy = accuracy_score(y_test, y_pred_test)
train_class_report = classification_report(y_train, y_pred_train)
test_class_report = classification_report(y_test, y_pred_test)

train_accuracy, test_accuracy, train_class_report, test_class_report
